### NOMIS API Data Extract for Historic Census Data.

The NOMIS RESTful API limits the size of the 'queries / calls' you can request.
For example, you cannot request all of the geography codes in a single call, the site will time-out or error due to the number of data items. Additionally each call is limited to a maximum of 25k records; hence the need for a few nested programming loops etc. etc. etc.

***Here are is a useful reading resource:***
https://www.nomisweb.co.uk/api/v01/help

***Here is the target data Darren Barnes has requested:***

"*Had a chat with census team this morning and this is the dataset they would like us to test out
Its a 2011 census table LC3409EW >> https://www.nomisweb.co.uk/census/2011/lc3409ew and includes General Health, Tenure and Age, covers England and Wales, and goes down to OA level.
obviously its available through the NOMIS API and i would like us to suck that into COGS rather than download and transform. The census team have had some very good feedback from Ahmed on the data viz and sparql and i think this has generated some buzz within the census senior management team. So the idea is to load this data into COGS and have Ahmed do a data viz on it and compare that with the CMD.
We have looked at NOMIS API already and i would like us to give consideration on what and when we could get this loaded into COGS."*

**OA**: Census **O**utput **A**reas.

In [1]:
# Components & Libraries:
import pandas as pd
import json
import slug
from pathlib import Path
import datetime
import numpy as np

from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter
from requests import Session
session = CacheControl(Session(), cache=FileCache('.cache'), heuristic=ExpiresAfter(days=7))

import re

pd.set_option('display.max_colwidth', -1)

absolute_start = datetime.datetime.now()

In [2]:
# NOMIS RESTful: Get ID for Dataset from NOMIS Data Families:

# Get Family List:
baseURL_ID = 'https://www.nomisweb.co.uk/api/v01/dataset/def.sdmx.json'
df_ID = pd.read_json(baseURL_ID)
display(df_ID)

i = 0
while True:
    str_df_ID = str(df_ID.structure[2]['keyfamily'][i]['name']['value'])
    if 'General health by tenure by age' in str_df_ID:
        print(str_df_ID + ' -- ID: ' + str(df_ID.structure[2]['keyfamily'][i]['id']))
        str_dataset_id = str(df_ID.structure[2]['keyfamily'][i]['id'])
        break
    i += 1

df_ID = pd.DataFrame() # Memory


structure
common          http://www.SDMX.org/resources/SDMXML/schemas/v2_0/common                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

LC3409EW - General health by tenure by age -- ID: NM_1404_1


In [3]:
# NOMIS RESTful: Get Parent Level Geography for Dataset:

# Get Geography sets using our ID:
baseURL_Geography = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography.def.sdmx.json'
df_Geography = pd.read_json(baseURL_Geography)
display(df_Geography)

i = 0
flag_england = False
flag_wales = False
while True:
    str_df_Geog_ID = str(df_Geography.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if (str_df_Geog_ID == 'England'):
        print(str_df_Geog_ID + ' -- ID: ' + str(df_Geography.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_ID_England = str(df_Geography.structure[0]['codelist'][0]['code'][i]['value'])
        flag_england = True
    if (str_df_Geog_ID == 'Wales'):
        print(str_df_Geog_ID + ' -- ID: ' + str(df_Geography.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_ID_Wales = str(df_Geography.structure[0]['codelist'][0]['code'][i]['value'])
        flag_wales = True
    if flag_england == True and flag_wales == True:
        break
    i += 1

df_Geography = pd.DataFrame() # Memory
    

,structure
codelists,"{'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_1404_1_GEOGRAPHY', 'code': [{'annotations': {'annotation': [{'annotationtext': 'countries', 'annotationtitle': 'TypeName'}, {'annotationtext': 499, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'K04000001', 'annotationtitle': 'GeogCode'}]}, 'description': {'value': 'England and Wales', 'lang': 'en'}, 'value': 2092957703}, {'annotations': {'annotation': [{'annotationtext': 'countries', 'annotationtitle': 'TypeName'}, {'annotationtext': 499, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'E92000001', 'annotationtitle': 'GeogCode'}]}, 'description': {'value': 'England', 'lang': 'en'}, 'value': 2092957699}, {'annotations': {'annotation': [{'annotationtext': 'countries', 'annotationtitle': 'TypeName'}, {'annotationtext': 499, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'W92000004', 'annotationtitle': 'GeogCode'}]}, 'parentcode': 2092957700, 'description': {'value': 'Wales', 'lang': 'en'}, 'value': 2092957700}], 'name': {'value': 'geography', 'lang': 'en'}, 'uri': ''}]}"
common,http://www.SDMX.org/resources/SDMXML/schemas/v2_0/common
header,"{'id': 'NM_1404_1', 'prepared': '2020-08-05T14:39:02Z', 'sender': {'contact': {'email': 'census.customerservices@ons.gov.uk', 'name': 'Census', 'telephone': '01329 444 972', 'uri': 'http://www.ons.gov.uk/census'}, 'id': 'NOMIS'}, 'test': 'false'}"
schemalocation,http://sdmx.org/docs/2_0/SDMXMessage.xsd
structure,http://www.SDMX.org/resources/SDMXML/schemas/v2_0/structure
xmlns,http://www.SDMX.org/resources/SDMXML/schemas/v2_0/message
xsi,http://www.w3.org/2001/XMLSchema-instance


England -- ID: 2092957699
Wales -- ID: 2092957700


In [4]:
# NOMIS RESTful: Get Hierarchical Geography for Dataset using Parent Geography IDs:

# Get Geography sets using Parent Level Geography IDs for WALES:
baseURL_Geography_Wales = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_ID_Wales + '.def.sdmx.json'
df_Geography_Wales = pd.read_json(baseURL_Geography_Wales)
display(df_Geography_Wales)

i = 0
while True:
    str_df_Geog_L1_ID = str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if '2011 output areas within Wales' == str_df_Geog_L1_ID:
        print(str_df_Geog_L1_ID + ' -- ID: ' + str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_L1_ID_Wales = str(df_Geography_Wales.structure[0]['codelist'][0]['code'][i]['value'])
        break
    i += 1
    
df_Geography_Wales = pd.DataFrame() # Memory


structure
codelists       {'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_1404_1_GEOGRAPHY', 'code': [{'annotations': {'annotation': [{'annotationtext': 'countries', 'annotationtitle': 'TypeName'}, {'annotationtext': 499, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'W92000004', 'annotationtitle': 'GeogCode'}]}, 'parentcode': 2092957700, 'description': {'value': 'Wales', 'lang': 'en'}, 'value': 2092957700}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 267, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'built-up areas including subdivisions', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700, 'annotationtitle': 'ParentCode'}, {'annotationtext': 531, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'built-up areas including subdivisions within Wales', 'lang': 'en'}, 'value': '2092957700TYPE267'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 269, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'built-up areas', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700, 'annotationtitle': 'ParentCode'}, {'annotationtext': 434, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'built-up areas within Wales', 'lang': 'en'}, 'value': '2092957700TYPE269'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 273, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'national assembly for wales electoral regions 2010', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700, 'annotationtitle': 'ParentCode'}, {'annotationtext': 5, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'national assembly for wales electoral regions 2010 within Wales', 'lang': 'en'}, 'value': '2092957700TYPE273'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 274, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'postcode areas', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700, 'annotationtitle': 'ParentCode'}, {'annotationtext': 5, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'postcode areas within Wales', 'lang': 'en'}, 'value': '2092957700TYPE274'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 275, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'postcode districts', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700, 'annotationtitle': 'ParentCode'}, {'annotationtext': 178, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'postcode districts within Wales', 'lang': 'en'}, 'value': '2092957700TYPE275'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 276, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'postcode sectors', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700, 'annotationtitle': 'ParentCode'}, {'annotationtext': 457, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'postcode sectors within Wales', 'lang': 'en'}, 'value': '2092957700TYPE276'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 277, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'national assembly for wales constituencies 2010', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700, 'annotationtitle': 'ParentCode'}, {'annotationtext': 40, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'national assembly for wales constituencies 2010 within Wales', 'lang': 'en'}, 'value': '2092957700TYPE277'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 279, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'parishes 2011', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957700,

2011 output areas within Wales -- ID: 2092957700TYPE299


In [5]:
# NOMIS RESTful: Get Hierarchical Geography for Dataset using Parent Geography IDs:

# Get Geography sets using Parent Level Geography IDs for England:
baseURL_Geography_England = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_ID_England + '.def.sdmx.json'
df_Geography_England = pd.read_json(baseURL_Geography_England)
display(df_Geography_England)

i = 0
while True:
    str_df_Geog_L1_ID = str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['description']['value'])
    if '2011 output areas within England' == str_df_Geog_L1_ID:
        print(str_df_Geog_L1_ID + ' -- ID: ' + str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['value']))
        str_df_Geog_L1_ID_England = str(df_Geography_England.structure[0]['codelist'][0]['code'][i]['value'])
        break
    i += 1
    
df_Geography_England = pd.DataFrame() # Memory


structure
codelists       {'codelist': [{'agencyid': 'NOMIS', 'id': 'CL_1404_1_GEOGRAPHY', 'code': [{'annotations': {'annotation': [{'annotationtext': 'countries', 'annotationtitle': 'TypeName'}, {'annotationtext': 499, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'E92000001', 'annotationtitle': 'GeogCode'}]}, 'description': {'value': 'England', 'lang': 'en'}, 'value': 2092957699}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 265, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'NHS area teams', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 25, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'NHS area teams within England', 'lang': 'en'}, 'value': '2092957699TYPE265'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 266, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'clinical commissioning groups', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 211, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'clinical commissioning groups within England', 'lang': 'en'}, 'value': '2092957699TYPE266'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 267, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'built-up areas including subdivisions', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 6091, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'built-up areas including subdivisions within England', 'lang': 'en'}, 'value': '2092957699TYPE267'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 269, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'built-up areas', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 5059, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'built-up areas within England', 'lang': 'en'}, 'value': '2092957699TYPE269'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 274, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'postcode areas', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 100, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'postcode areas within England', 'lang': 'en'}, 'value': '2092957699TYPE274'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 275, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'postcode districts', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 2091, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'postcode districts within England', 'lang': 'en'}, 'value': '2092957699TYPE275'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 276, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'postcode sectors', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 7578, 'annotationtitle': 'ChildCount'}]}, 'description': {'value': 'postcode sectors within England', 'lang': 'en'}, 'value': '2092957699TYPE276'}, {'annotations': {'annotation': [{'annotationtext': 'True', 'annotationtitle': 'IsAbstractCode'}, {'annotationtext': 279, 'annotationtitle': 'TypeCode'}, {'annotationtext': 'parishes 2011', 'annotationtitle': 'TypeName'}, {'annotationtext': 2092957699, 'annotationtitle': 'ParentCode'}, {'annotationtext': 10490, 'annotationtitle': 'ChildCount'}]}, 'description'

2011 output areas within England -- ID: 2092957699TYPE299


In [6]:
# NOMIS RESTful: Get Level 2 Geography Codes for Dataset using Level 1 Geography IDs:

# Get Geography sets using Level 1 Geography IDs for WALES:
baseURL_Geography_Wales_L1 = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_L1_ID_Wales + '.def.sdmx.json'
df_Geography_Wales_L2 = pd.read_json(baseURL_Geography_Wales_L1)
#display(df_Geography_Wales_L2)

i = 0
str_df_Geog_L2_ID_Wales_LowerRange = 'W99999999999999999'
str_df_Geog_L2_ID_Wales_HigherRange = ''
while True:
    try:
        str_df_Geog_L2_ID = str(df_Geography_Wales_L2.structure[0]['codelist'][0]['code'][i]['description']['value'])
        if str_df_Geog_L2_ID < str_df_Geog_L2_ID_Wales_LowerRange:
            str_df_Geog_L2_ID_Wales_LowerRange = str_df_Geog_L2_ID
        if str_df_Geog_L2_ID > str_df_Geog_L2_ID_Wales_HigherRange:
            str_df_Geog_L2_ID_Wales_HigherRange = str_df_Geog_L2_ID       
        i += 1
  
    except IndexError:
        break

df_Geography_Wales_L2 = pd.DataFrame() # Memory


In [7]:
# NOMIS RESTful: Get Level 2 Geography Codes for Dataset using Level 1 Geography IDs:

# Get Geography sets using Level 1 Geography IDs for ENGLAND:
baseURL_Geography_England_L1 = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '/geography/' + str_df_Geog_L1_ID_England + '.def.sdmx.json'
df_Geography_England_L2 = pd.read_json(baseURL_Geography_England_L1)
#display(df_Geography_England_L2)

i = 0
str_df_Geog_L2_ID_England_LowerRange = 'E99999999999999999'
str_df_Geog_L2_ID_England_HigherRange = ''
while True:
    try:
        str_df_Geog_L2_ID = str(df_Geography_England_L2.structure[0]['codelist'][0]['code'][i]['description']['value'])
        if str_df_Geog_L2_ID < str_df_Geog_L2_ID_England_LowerRange:
            str_df_Geog_L2_ID_England_LowerRange = str_df_Geog_L2_ID
        if str_df_Geog_L2_ID > str_df_Geog_L2_ID_England_HigherRange:
            str_df_Geog_L2_ID_England_HigherRange = str_df_Geog_L2_ID       
        i += 1
        
    except IndexError:
        break
    
df_Geography_England_L2 = pd.DataFrame() # Memory


In [8]:
print(str_df_Geog_L2_ID_Wales_LowerRange)
print(str_df_Geog_L2_ID_Wales_HigherRange)
print(str_df_Geog_L2_ID_England_LowerRange)
print(str_df_Geog_L2_ID_England_HigherRange)

W00000001
W00010265
E00000001
E00176774


In [9]:
# set explicit types to speed things up
dtype={'DATE_NAME':np.int,
        'GEOGRAPHY_CODE': str,
        'GEOGRAPHY_TYPE': str,
        'C_TENHUK11_NAME':  str,
        'C_AGE_NAME': str,
        'C_HEALTH_NAME': str,
        'OBS_VALUE':np.float,
        'RECORD_OFFSET':np.int,
        'RECORD_COUNT':np.int}

In [10]:
# NOMIS RESTful: Get Tabular Data using Level 2 Geography Code IDs:

# Get Data for WALES:
#baseURL_Geography_Wales_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_Wales_LowerRange + '...' + str_df_Geog_L2_ID_Wales_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_name,geography_code,c_tenhuk11_name,c_age_name,c_health_name,measures_name,obs_value,obs_status_name,record_offset,record_count'
# Revised:
baseURL_Geography_Wales_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_Wales_LowerRange + '...' + str_df_Geog_L2_ID_Wales_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_code,geography_type,c_tenhuk11_name,c_age_name,c_health_name,obs_value,record_offset,record_count'
print(baseURL_Geography_Wales_Data)
stream = session.get(baseURL_Geography_Wales_Data, stream=True).raw
stream.decode_content = True
final_df_wales = pd.read_csv(stream)

additionalURL = ''
intRecordController = 0
    
# The links below are limited to the first 25,000 cells per call.
while True:
    start = datetime.datetime.now()
    additionalURL = '&RecordOffset=' + str(intRecordController)
    concatenatedURL = baseURL_Geography_Wales_Data + additionalURL
    stream = session.get(concatenatedURL, stream=True).raw
    stream.decode_content = True
    
    dataframe = pd.read_csv(stream, engine='c', na_filter=False, dtype=dtype)

    if (dataframe.empty):
        break
    
    #display(dataframe)
    # Additional code due to large datasets:
    # For a very basic progress monitor (no fancy code this time round):
    print("Just received record:" + (dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) + ' of ' + (dataframe.tail(1).RECORD_COUNT.to_string(index=False)) + ' >>> ' + str(round((int(dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) / int(dataframe.tail(1).RECORD_COUNT.to_string(index=False)) * 100),2)) + '%... in ' + str(datetime.datetime.now() - start))
    frames = [final_df_wales, dataframe]
    final_df_wales = pd.concat(frames)
    
    intRecordController = intRecordController + 25000

    
final_df_wales.GEOGRAPHY_TYPE  = final_df_wales.GEOGRAPHY_TYPE.apply(slug.slug)
final_df_wales.C_TENHUK11_NAME = final_df_wales.C_TENHUK11_NAME.apply(slug.slug)
final_df_wales.C_HEALTH_NAME   = final_df_wales.C_HEALTH_NAME.apply(slug.slug)
final_df_wales['C_AGE_NAME']   = final_df_wales['C_AGE_NAME'].replace('Age 0 to 15', 'Y0T15').replace('Age 16 to 49', 'Y16T49').replace('Age 50 to 64', 'Y50T64').replace('Age 65 and over', 'Y_GE65') 

display(final_df_wales)


https://www.nomisweb.co.uk/api/v01/dataset/NM_1404_1.data.csv?date=latest&geography=W00000001...W00010265&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_code,geography_type,c_tenhuk11_name,c_age_name,c_health_name,obs_value,record_offset,record_count
Just received record: 24999 of  481728 >>> 5.19%... in 0:00:00.064140
Just received record: 49999 of  481728 >>> 10.38%... in 0:00:00.065660
Just received record: 74999 of  481728 >>> 15.57%... in 0:00:00.064494
Just received record: 99999 of  481728 >>> 20.76%... in 0:00:00.064521
Just received record: 124999 of  481728 >>> 25.95%... in 0:00:00.069104
Just received record: 149999 of  481728 >>> 31.14%... in 0:00:00.071079
Just received record: 174999 of  481728 >>> 36.33%... in 0:00:00.063270
Just received record: 199999 of  481728 >>> 41.52%... in 0:00:00.069351
Just received record: 224999 of  481728 >>> 46.71%... in 0:00:00.071277
Just received record: 249999 of  481728 >>> 51.9%... in 0:00:00.0

,DATE_NAME,GEOGRAPHY_CODE,GEOGRAPHY_TYPE,C_TENHUK11_NAME,C_AGE_NAME,C_HEALTH_NAME,OBS_VALUE,RECORD_OFFSET,RECORD_COUNT
0,2011,W00000001,2011-output-areas,owned-owned-outright,Y0T15,very-good-or-good-health,9.0,0,481728
1,2011,W00000001,2011-output-areas,owned-owned-outright,Y0T15,fair-health,0.0,1,481728
2,2011,W00000001,2011-output-areas,owned-owned-outright,Y0T15,bad-or-very-bad-health,1.0,2,481728
3,2011,W00000001,2011-output-areas,owned-owned-outright,Y16T49,very-good-or-good-health,21.0,3,481728
4,2011,W00000001,2011-output-areas,owned-owned-outright,Y16T49,fair-health,3.0,4,481728
5,2011,W00000001,2011-output-areas,owned-owned-outright,Y16T49,bad-or-very-bad-health,0.0,5,481728
6,2011,W00000001,2011-output-areas,owned-owned-outright,Y50T64,very-good-or-good-health,30.0,6,481728
7,2011,W00000001,2011-output-areas,owned-owned-outright,Y50T64,fair-health,4.0,7,481728
8,2011,W00000001,2011-output-areas,owned-owned-outright,Y50T64,bad-or-very-bad-health,1.0,8,481728
9,2011,W00000001,2011-output-areas,owned-owned-outright,Y_GE65,very-good-or-good-health,28.0,9,481728


In [11]:
final_df_wales.to_csv('final_df_wales.csv.gz', date_format='%Y%m%d', compression="gzip")
final_df_wales = pd.DataFrame() # Memory

In [13]:
# NOMIS RESTful: Get Tabular Data using Level 2 Geography Code IDs:

# Get Data for ENGLAND:
#baseURL_Geography_England_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_England_LowerRange + '...' + str_df_Geog_L2_ID_England_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_name,geography_code,c_tenhuk11_name,c_age_name,c_health_name,measures_name,obs_value,obs_status_name,record_offset,record_count'
# Revised:
baseURL_Geography_England_Data = 'https://www.nomisweb.co.uk/api/v01/dataset/' + str_dataset_id + '.data.csv?date=latest&geography=' + str_df_Geog_L2_ID_England_LowerRange + '...' + str_df_Geog_L2_ID_England_HigherRange + '&c_tenhuk11=2,3,5,6&c_age=1...4&c_health=1...3&measures=20100&select=date_name,geography_code,geography_type,c_tenhuk11_name,c_age_name,c_health_name,obs_value,record_offset,record_count'
stream = session.get(baseURL_Geography_England_Data, stream=True).raw
stream.decode_content = True
final_df_england = pd.read_csv(stream)

additionalURL = ''
intRecordController = 0
# The links below are limited to the first 25,000 cells per call.
while True:
    start = datetime.datetime.now()
    additionalURL = '&RecordOffset=' + str(intRecordController)
    concatenatedURL = baseURL_Geography_England_Data + additionalURL
    stream = session.get(concatenatedURL, stream=True).raw
    stream.decode_content = True
    
    dataframe = pd.read_csv(stream, engine='c', na_filter=False, dtype=dtype)
    
    if (dataframe.empty):
        break
    
    #display(dataframe)
    # Additional code due to large datasets:
    # For a very basic progress monitor (no fancy code this time round):
    try:
        print("Just received record:" + (dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) + ' of ' + (dataframe.tail(1).RECORD_COUNT.to_string(index=False)) + ' >>> ' + str(round((int(dataframe.tail(1).RECORD_OFFSET.to_string(index=False)) / int(dataframe.tail(1).RECORD_COUNT.to_string(index=False)) * 100),2)) + '%... in ' + str(datetime.datetime.now() - start))
    except ValueError:
        pass # avoid potential casting error on last call
    frames = [final_df_england, dataframe]
    final_df_england = pd.concat(frames)
    intRecordController = intRecordController + 25000

#display(final_df_england)

Just received record: 24999 of  8225856 >>> 0.3%... in 0:00:00.092907
Just received record: 49999 of  8225856 >>> 0.61%... in 0:00:00.090055
Just received record: 74999 of  8225856 >>> 0.91%... in 0:00:00.098998
Just received record: 99999 of  8225856 >>> 1.22%... in 0:00:00.089838
Just received record: 124999 of  8225856 >>> 1.52%... in 0:00:00.091381
Just received record: 149999 of  8225856 >>> 1.82%... in 0:00:00.089679
Just received record: 174999 of  8225856 >>> 2.13%... in 0:00:00.090779
Just received record: 199999 of  8225856 >>> 2.43%... in 0:00:00.086084
Just received record: 224999 of  8225856 >>> 2.74%... in 0:00:00.086446
Just received record: 249999 of  8225856 >>> 3.04%... in 0:00:00.100093
Just received record: 274999 of  8225856 >>> 3.34%... in 0:00:00.092542
Just received record: 299999 of  8225856 >>> 3.65%... in 0:00:00.102110
Just received record: 324999 of  8225856 >>> 3.95%... in 0:00:00.103660
Just received record: 349999 of  8225856 >>> 4.25%... in 0:00:00.0880

In [14]:
final_df_england.to_csv('final_df_england.csv.gz', date_format='%Y%m%d', compression="gzip")
final_df_england = pd.DataFrame() # Memory

In [ ]:
#display(df_Geography_England_L2)

In [ ]:
#df_Geography_England_L2.to_csv('final_df_england_geographies.csv', date_format='%Y%m%d')

In [16]:
print("Completion time for complete script is: ", datetime.datetime.now() - absolute_start)

Completion time for complete script is:  0:21:15.990342
